# Step 1 - Load Data

In [1]:
# Import Packages
import pandas as pd
import numpy as np

In [ ]:
# Load Data
client_df = pd.read_csv('clean_data_after_eda.csv')
price_df = pd.read_csv('price_data.csv')

In [3]:
# Convert date column to datetime format
client_df['date_activ'] = pd.to_datetime(client_df['date_activ'])
client_df['date_end'] = pd.to_datetime(client_df['date_end'])
client_df['date_modif_prod'] = pd.to_datetime(client_df['date_modif_prod'])
client_df['date_renewal'] = pd.to_datetime(client_df['date_renewal'])
price_df['price_date'] = pd.to_datetime(price_df['price_date'])

In [4]:
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              14606 non-null  object        
 1   channel_sales                   14606 non-null  object        
 2   cons_12m                        14606 non-null  int64         
 3   cons_gas_12m                    14606 non-null  int64         
 4   cons_last_month                 14606 non-null  int64         
 5   date_activ                      14606 non-null  datetime64[ns]
 6   date_end                        14606 non-null  datetime64[ns]
 7   date_modif_prod                 14606 non-null  datetime64[ns]
 8   date_renewal                    14606 non-null  datetime64[ns]
 9   forecast_cons_12m               14606 non-null  float64       
 10  forecast_cons_year              14606 non-null  int64         
 11  fo

In [5]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193002 entries, 0 to 193001
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  193002 non-null  object        
 1   price_date          193002 non-null  datetime64[ns]
 2   price_off_peak_var  193002 non-null  float64       
 3   price_peak_var      193002 non-null  float64       
 4   price_mid_peak_var  193002 non-null  float64       
 5   price_off_peak_fix  193002 non-null  float64       
 6   price_peak_fix      193002 non-null  float64       
 7   price_mid_peak_fix  193002 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 11.8+ MB


# Step 2 - Dec-Jan Off-Peak Price Difference

In [6]:
# Group prices by ID and Month (take the average if there are duplicates per month)
monthly_prices = price_df.groupby(['id', 'price_date']).agg({
    'price_off_peak_var': 'mean',  # Energy Price (Variable)
    'price_off_peak_fix': 'mean'   # Power Price (Fixed)
}).reset_index()

In [7]:
# Take the January and December prices
jan_prices = monthly_prices[monthly_prices['price_date'] == '2015-01-01']
dec_prices = monthly_prices[monthly_prices['price_date'] == '2015-12-01']

In [8]:
# Combine to calculate the difference
diff_df = pd.merge(dec_prices, jan_prices, on='id', suffixes=('_dec', '_jan'))
diff_df['offpeak_diff_dec_january_energy'] = diff_df['price_off_peak_var_dec'] - diff_df['price_off_peak_var_jan']
diff_df['offpeak_diff_dec_january_power'] = diff_df['price_off_peak_fix_dec'] - diff_df['price_off_peak_fix_jan']

In [9]:
# Take only the relevant columns to merge into the main data
price_features = diff_df[['id', 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power']]

In [10]:
# Merge to client_df
client_df = pd.merge(client_df, price_features, on='id', how='left')

# Step 3 - Creative Feature Engineering

In [11]:
# A. Tenure (Subscription Duration)
# Calculating how many years a customer has been active
client_df['tenure'] = (client_df['date_end'] - client_df['date_activ']).dt.days / 365.25

In [12]:
# B. Months Since Last Modification
# Calculating how many months it has been since the customer last changed their contract
client_df['months_since_last_modif'] = (client_df['date_end'] - client_df['date_modif_prod']).dt.days / 30

In [13]:
# C. Log Transformation for Consumption Data
# Because the consumption data is highly skewed, we use a log transformation.
# We use log1p (log(x+1)) to avoid errors if the value is 0.
cols_to_log = ['cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_12m']
for col in cols_to_log:
    client_df[f'log_{col}'] = np.log1p(client_df[col])

# Step 4 - Check Result

In [14]:
print("Latest Data Info")
print(client_df.info())

Latest Data Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14606 entries, 0 to 14605
Data columns (total 52 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   id                               14606 non-null  object        
 1   channel_sales                    14606 non-null  object        
 2   cons_12m                         14606 non-null  int64         
 3   cons_gas_12m                     14606 non-null  int64         
 4   cons_last_month                  14606 non-null  int64         
 5   date_activ                       14606 non-null  datetime64[ns]
 6   date_end                         14606 non-null  datetime64[ns]
 7   date_modif_prod                  14606 non-null  datetime64[ns]
 8   date_renewal                     14606 non-null  datetime64[ns]
 9   forecast_cons_12m                14606 non-null  float64       
 10  forecast_cons_year               14606 no

In [16]:
print("Top 5 Data Examples (New Feature")
client_df[['id', 'offpeak_diff_dec_january_energy', 'tenure', 'log_cons_12m']].head()

Top 5 Data Examples (New Feature


,id,offpeak_diff_dec_january_energy,tenure,log_cons_12m
0,24011ae4ebbe3035111d65fa7c15bc57,0.020057,3.000684,0.000000
1,d29c2c54acc38ff3c0614d0a653813dd,-0.003767,7.025325,8.446985
2,764c75f661154dac3a6c254cd082ea7d,-0.004670,6.001369,6.300786
3,bba03439a292a1e166f80264c16191cb,-0.004547,6.001369,7.368340
4,149d57cf92fc41cf94415803a877cb4b,-0.006192,6.146475,8.395252


**Brief Description of Features Created:**
* `offpeak_diff_dec_january_energy`: Measuring whether prices have risen or fallen from the beginning to the end of the year. Significant price increases can trigger churn.
* `tenure`: Existing customers may be more loyal (less likely to churn) than new customers.
* `months_since_last_modif`: Customers who have just renewed their contracts are less likely to switch.
* `log_cons_...`: Helping models handle extreme differences in scale in consumption data (e.g., better distinguishing between small MSMEs and large factories).
